In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Saving model checkpoints after each epoch
os.mkdir("/kaggle/working/models/2pathcnn_bilstm")

In [ ]:
import tensorflow as tf
from keras.layers import Conv3D, ConvLSTM3D, Reshape, Conv3DTranspose, UpSampling3D, MaxPooling3D, Concatenate, Input
from keras.layers import Bidirectional, BatchNormalization, Dropout, Dense, Flatten, Attention, Add, Activation
from keras.models import Model
import keras
import matplotlib.pyplot as plt
from keras.metrics import categorical_crossentropy
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam,Adadelta
from glob import glob


In [ ]:
#Get corresponsing Mask id
def get_id(path):
    slice = path.split(".")[2]
    id = slice[len(slice)-1:len(slice)-4:-1]

    return id[-1::-1]

In [ ]:
from sklearn.decomposition import PCA,IncrementalPCA
def pca_preprocessing(sample):
    #divide into its channels
    t1ce,t2,flair=sample[:,:,:,0],sample[:,:,:,1],sample[:,:,:,2]
    t1ce_f=np.reshape(t1ce,(128,128*128))
    t2_f=np.reshape(t2,(128,128*128))
    flair_f=np.reshape(flair,(128,128*128))
    
    p=PCA()
    #Get components that explains 99% of the slice
    #t1ce
    p.fit(t1ce_f)

    var1=np.cumsum(p.explained_variance_ratio_)*100
    k1=np.argmax(var1>99)
    
    #t2
    p.fit(t2_f)
    var2=np.cumsum(p.explained_variance_ratio_)*100
    k2=np.argmax(var2>99)
    
    #flair
    p.fit(flair_f)
    var3=np.cumsum(p.explained_variance_ratio_)*100
    k3=np.argmax(var3>99)
    
    #perform pca for k1,k2,k3 components
    pca_t1ce=PCA(k1)
    t1ce_t=pca_t1ce.inverse_transform(pca_t1ce.fit_transform(t1ce_f))
    
    pca_t2=PCA(k2)
    t2_t=pca_t2.inverse_transform(pca_r2.fit_transform(t2_f))
    
    pca_flair=PCA(k3)
    flair_t=pca_flair.inverse_transform(pca_flair.fit_transform(flair_f))
    
    #rebuild the image
    t1ceR=np.reshape(t1ce_t,(128,128,128))
    t2R=np.reshape(t2_t,(128,128,128))
    flairR=np.reshape(flair_t,(128,128,128))
    
    #Stack the channels
    sample_reduced=np.stack([t1ceR,t2R,flairR])
    
    return sample_reduced
    

In [ ]:
#Load all helper functions onto notebook
#Image generator
def generate_image(paths):
    for img_path in paths:
        example_id = get_id(img_path)
        sample = np.load(img_path)

        mask = np.load("../input/mri-data/Data/Train/masks_reformatted/mask_"+example_id+".npy","r")
        # reshape mask
        # mask_reshaped=np.reshape(mask,(128*128*128,4))
        #pca
        sample=pca_preprocessing(sample)
        
        # Normalization
        normalized_sample = (sample-np.mean(sample, axis=0))/np.std(sample, axis=0)
        #normalized_sample=np.round(normalized_sample,3)
        yield normalized_sample, mask


In [ ]:
#Batch generator

def batch_generator(paths, batch_size):
    while True:
        image = generate_image(paths)

        img_batch = list()
        mask_batch = list()

        for sample, mask in image:
            img_batch.append(sample)
            mask_batch.append(mask)

            if (len(img_batch) == batch_size):
                yield np.stack(img_batch, axis=0), np.stack(mask_batch, axis=0)
                img_batch = []
                mask_batch = []

        if (len(img_batch) != 0):
            yield np.stack(img_batch, axis=0), np.stack(mask_batch, axis=0)
            img_batch = []
            mask_batch = []

In [ ]:
#Metrics 
from keras.losses import categorical_crossentropy
def dice_coef(y_true, y_pred):
    ytrue_f = keras.backend.flatten(y_true)
    ypred_f = keras.backend.flatten(y_pred)

    intersection = keras.backend.sum(
        ytrue_f*ypred_f)

    E = keras.backend.epsilon()
    dice = (2.*intersection+E)/(keras.backend.sum(ytrue_f)+keras.backend.sum(ypred_f)+E)
    return dice


def dice_loss(y_true, y_pred):
    dice_loss = 1-dice_coef(y_true, y_pred)
    return dice_loss


def overall_loss(y_true, y_pred):
    loss = (0.5*categorical_crossentropy(y_true, y_pred)) + \
        dice_loss(y_true, y_pred)
    return loss

def multilabel_loss(y_true,y_pred,m):
    dice=0
    for i in range(m):
        dice+=dice_coef(y_true[:,:,:,i],y_pred[:,:,:,i])
    return dice


In [ ]:
#Learning rate scheduler
def scheduler(epoch,lr):
    if(epoch%5==0):
        return lr*tf.math.exp(-0.1*epoch)
    else:
        return lr

In [ ]:
#Add augumentation layers
#Rotation at random

import random
from scipy import ndimage

@tf.function
def rotate(slice,angle):
    def srotate(slice,angle):
        vol=ndimage.rotate(slice,angles,reshape=False)
        return vol
    augumentation=tf.numpy_function(srotate,[slice],tf.float32)
    return augumentation

In [ ]:
auguemtation=tf.keras.Sequential(
    rotate=tf.keras.Layers.Lambda(lambda x:rotate(x,angle=30))
)

In [ ]:
#2 path CNN model
def create_2pathed_shallow():
    input_layer = Input((128, 128, 128, 3))''
    # first encoding block
    # Convolution layers
    conv1_e_1 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(input_layer)
    conv1_e_2 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv1_e_1)
    conv1_e_3 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv1_e_2)

    # Downsample
    pool1_e_1 = MaxPooling3D(pool_size=(2, 2, 2), strides=2)(conv1_e_3)

    # Convolution Layers
    conv1_e_4 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(pool1_e_1)
    conv1_e_5 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv1_e_4)
    conv1_e_6 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), strides=1, activation='elu', use_bias=True, bias_initializer='he_normal')(conv1_e_5)

    # Batch normalization to get mean 0 activations
    batch1 = BatchNormalization()(conv1_e_6)

    # Second encoding block
    # Convolutions
    conv2_e_1 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(input_layer)
    conv2_e_2 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_1)
    conv2_e_3 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_2)

    # Pooling
    pool2_e_1 = MaxPooling3D(pool_size=(2, 2, 2), strides=2)(conv2_e_3)

    # Convolutions
    conv2_e_4 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(pool2_e_1)
    conv2_e_5 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_4)
    conv2_e_6 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_5)

    batch2 = BatchNormalization()(conv2_e_6)

    # Concatenate
    path_combined = Concatenate(axis=4)([batch1, batch2])

    # Combined convolutions
    conv_7 = Conv3D(filters=32, kernel_initializer='he_normal', kernel_size=(3, 3, 3), activation='elu',
                    strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(path_combined)
    conv_8 = Conv3D(filters=64, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv_7)
    conv_9 = Conv3D(filters=128, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv_8)

    batch3 = BatchNormalization()(conv_9)

    # Deconvolution
    d_conv = Conv3DTranspose(filters=128, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, use_bias=True, bias_initializer='he_normal')(batch3)
    d_conv_1 = Conv3DTranspose(filters=64, kernel_initializer='he_normal', activation='elu',
                               strides=1, kernel_size=(3, 3, 3), padding='valid', use_bias=True, bias_initializer='he_normal')(d_conv)
    d_conv_2 = Conv3DTranspose(filters=32, kernel_initializer='he_normal', activation='elu',
                               strides=1, kernel_size=(3, 3, 3), padding='valid', use_bias=True, bias_initializer='he_normal')(d_conv_1)

    # first decoding block
    dconv1_e_1 = Conv3DTranspose(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(d_conv_2)
    dconv1_e_2 = Conv3DTranspose(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(dconv1_e_1)
    dconv1_e_3 = Conv3DTranspose(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(dconv1_e_2)

    upsample_e_1 = UpSampling3D(size=(2, 2, 2))(dconv1_e_3)

    dconv1_e_4 = Conv3DTranspose(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(upsample_e_1)
    dconv1_e_5 = Conv3DTranspose(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(dconv1_e_4)

    batchnorm = BatchNormalization()(dconv1_e_5)

    mask = Conv3DTranspose(filters=4, kernel_initializer='glorot_normal', kernel_size=(
        3, 3, 3), activation='softmax', strides=1, padding='valid', use_bias=True, bias_initializer='glorot_normal')(batchnorm)

    model = Model(inputs=input_layer, outputs=mask)
    return model


In [ ]:
#Compute steps per epoch
def steps(m,batchsize):
    return (m+batchsize-1)//batchsize

In [ ]:
#Main program
sample_paths=sorted(glob("../input/mri-data/Data/Train/samples_processed/*.npy"))
masks_paths=sorted(glob("../input/mri-data/Data/Train/masks_reformatted/*.npy"))


In [ ]:
#Train-test split
train_sample_paths,test_sample_paths=train_test_split(sample_paths,test_size=0.25)

In [ ]:
len(train_sample_paths)

In [ ]:
traingen=batch_generator(train_sample_paths,8)
testgen=batch_generator(test_sample_paths,8)

In [ ]:
sample,masks=next(testgen)

In [ ]:
sample.shape,masks.shape

In [ ]:
model=create_2pathed_shallow()


In [ ]:
model.summary()

In [ ]:
train_steps=steps(len(train_sample_paths),8)
test_steps=steps(len(test_sample_paths),8)

In [ ]:
import datetime

opt=tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt,loss=dice_loss,metrics=[dice_coef,categorical_crossentropy,overall_loss,tf.keras.metrics.CategoricalAccuracy()])

#callbacks
log_dir = "/kaggle/working/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
schedule=tf.keras.callbacks.LearningRateScheduler(scheduler)
plateau_handler=tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',patience=3,factor=0.1)
nanterminate=tf.keras.callbacks.TerminateOnNaN()
#earlystop=tf.keras.callbacks.EarlyStopping(monitor='val_loss')
checkpoints=tf.keras.callbacks.ModelCheckpoint(filepath="/kaggle/working/models/2pathcnn_bilstm",save_weights_only=False,save_freq='epoch',save_best_only=True,monitor='dice_coef',mode='max',verbose=1)

In [ ]:
tf.config.list_physical_devices()

In [ ]:
tf.test.is_gpu_available()

In [ ]:
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))


In [ ]:
#session.close()

In [ ]:
history=model.fit(traingen,steps_per_epoch=train_steps,epochs=30,validation_data=testgen,validation_steps=test_steps,verbose=1,callbacks=[tensorboard_callback,plateau_handler,schedule,checkpoints,nanterminate])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title("Loss curve for train")
plt.xlabel("Epochs")
plt.ylabel("Loss")


In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Loss curve for train")
plt.xlabel("Epochs")
plt.ylabel("Loss")

In [ ]:
plt.plot(history.history['val_dice_coef'])
plt.plot(history.history['dice_coef'])
plt.xlabel("Epochs")
plt.ylabel("Dice score")

In [ ]:
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.xlabel("Epochs")
plt.ylabel("Accuracy")

In [ ]:
plt.plot(history.history['overall_loss'])
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Weighted categorical and dice loss")